In [196]:
import pandas as pd
import numpy as np
from datetime import datetime
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [197]:
df = pd.read_excel('a1_FlightFare_Dataset.xlsx')

In [198]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [199]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

In [200]:
df.describe()

,Price
count,10001.000000
mean,9087.542346
std,4591.173973
min,1759.000000
25%,5276.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [201]:
df.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

Preprocessing 

In [202]:
for col in df.columns:
    if df[col].dtypes == 'object':
        print(col, df[col].nunique())
        print('--------------------------------')

Airline 12
--------------------------------
Date_of_Journey 44
--------------------------------
Source 5
--------------------------------
Destination 6
--------------------------------
Route 128
--------------------------------
Dep_Time 220
--------------------------------
Arrival_Time 1301
--------------------------------
Duration 364
--------------------------------
Total_Stops 5
--------------------------------
Additional_Info 9
--------------------------------


In [203]:
for col in ['Airline', 'Source', 'Destination', 'Additional_Info', 'Total_Stops']:
    print(col, df[col].value_counts(normalize=True) * 100)
    print('------------------------------------------')

Airline Jet Airways                          35.976402
IndiGo                               19.268073
Air India                            16.338366
Multiple carriers                    11.288871
SpiceJet                              7.689231
Vistara                               4.469553
Air Asia                              2.959704
GoAir                                 1.789821
Multiple carriers Premium economy     0.129987
Jet Airways Business                  0.049995
Vistara Premium economy               0.029997
Trujet                                0.009999
Name: Airline, dtype: float64
------------------------------------------
Source Delhi       42.575742
Kolkata     26.777322
Banglore    20.577942
Mumbai       6.469353
Chennai      3.599640
Name: Source, dtype: float64
------------------------------------------
Destination Cochin       42.575742
Banglore     26.777322
Delhi        11.908809
New Delhi     8.669133
Hyderabad     6.469353
Kolkata       3.599640
Name: Destinatio

In [204]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
9996,Multiple carriers,27/06/2019,Delhi,Cochin,DEL → IDR → BOM → COK,15:05,01:30 28 Jun,10h 25m,2 stops,No info,15965
9997,Jet Airways,6/05/2019,Kolkata,Banglore,CCU → BOM → BLR,06:30,18:15,11h 45m,1 stop,In-flight meal not included,10844
9998,Air India,12/05/2019,Banglore,Delhi,BLR → DEL,13:20,16:10,2h 50m,non-stop,No info,6121
9999,IndiGo,24/04/2019,Banglore,Delhi,BLR → DEL,22:10,01:00 25 Apr,2h 50m,non-stop,No info,3625


In [205]:
# Check null values
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [206]:
# Remove null values
df.dropna(inplace=True)

In [207]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [208]:
# Transform the 'Date_of_Journey' to a datetime object
df['Date_of_Journey'] = pd.to_datetime(df['Date_of_Journey'], format='%d/%m/%Y')

# Extract day, month, and year into separate columns
df['journey_day'] = df['Date_of_Journey'].dt.day
df['journey_month'] = df['Date_of_Journey'].dt.month

In [209]:
# Remove 'Date_of_Journey' column
df.drop(['Date_of_Journey'], axis=1, inplace=True)

In [210]:
# Transform 'Dep_Time' column to type date 
df['Dep_Time'] = pd.to_datetime(df['Dep_Time'], format='%H:%M')

# Extract hour and minutes into separate columns
df['dep_hour'] = df['Dep_Time'].dt.hour
df['dep_min'] = df['Dep_Time'].dt.minute

In [211]:
# Define a function to extract the first 5 characters from a string
def trunct(x):
    return x[:5]

df['Arrival_Time'] = df['Arrival_Time'].apply(trunct)

In [212]:
# Transform 'Arrival_Time' column to type date 
df['Arrival_Time'] = pd.to_datetime(df['Arrival_Time'], format='%H:%M') 

# Extract hour and minutes into separate columns
df['Arrival_hour'] = df['Arrival_Time'].dt.hour
df['Arrival_min'] = df['Arrival_Time'].dt.minute

In [213]:
df.drop(['Arrival_Time', 'Dep_Time'], axis=1, inplace=True)

In [214]:
df.head()

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month,dep_hour,dep_min,Arrival_hour,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1,5,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,9,6,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,12,5,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,1,3,16,50,21,35


In [215]:
df.columns

Index(['Airline', 'Source', 'Destination', 'Route', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price', 'journey_day', 'journey_month', 'dep_hour',
       'dep_min', 'Arrival_hour', 'Arrival_min'],
      dtype='object')

In [216]:
for col in ['Airline', 'Source', 'Destination', 'Additional_Info', 'Total_Stops']:
    print(col, df[col].value_counts(normalize=True) * 100)
    print('------------------------------------------')

Airline Jet Airways                          35.98
IndiGo                               19.27
Air India                            16.33
Multiple carriers                    11.29
SpiceJet                              7.69
Vistara                               4.47
Air Asia                              2.96
GoAir                                 1.79
Multiple carriers Premium economy     0.13
Jet Airways Business                  0.05
Vistara Premium economy               0.03
Trujet                                0.01
Name: Airline, dtype: float64
------------------------------------------
Source Delhi       42.57
Kolkata     26.78
Banglore    20.58
Mumbai       6.47
Chennai      3.60
Name: Source, dtype: float64
------------------------------------------
Destination Cochin       42.57
Banglore     26.78
Delhi        11.91
New Delhi     8.67
Hyderabad     6.47
Kolkata       3.60
Name: Destination, dtype: float64
------------------------------------------
Additional_Info No info        

In [217]:
# In the 'Destination' column we have 'Delhi' and 'New Delhi' wchich are the same destination. So, we need to transform it to one value
def transform_value(x):
    if x == 'New Delhi':
        x = 'Delhi'
    return x
df['Destination'] = df['Destination'].apply(transform_value)

In [218]:
df['Airline'].value_counts()

Jet Airways                          3598
IndiGo                               1927
Air India                            1633
Multiple carriers                    1129
SpiceJet                              769
Vistara                               447
Air Asia                              296
GoAir                                 179
Multiple carriers Premium economy      13
Jet Airways Business                    5
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [219]:
# Group "Multiple carriers Premium economy", "Jet Airways Business", "Vistara Premium economy " and "Trujet" into a single value called "other"
current_airline_values = df['Airline'].values.tolist()
new_airline_values = list()
for val in current_airline_values:
    if val not in ['Jet Airways', 'IndiGo', 'Air India', 'SpiceJet', 'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia']:
        val = 'other'
        new_airline_values.append(val)
        continue
    new_airline_values.append(val)

df['Airline'] = new_airline_values

In [220]:
print

<function print>

In [221]:
# Apply one-hot-encoding
df = pd.get_dummies(df, columns=['Airline', 'Source', 'Destination'], drop_first=True)

In [222]:
df.head()

,Route,Duration,Total_Stops,Additional_Info,Price,journey_day,journey_month,dep_hour,dep_min,Arrival_hour,...,Airline_Vistara,Airline_other,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata
0,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,22,20,1,...,0,0,0,0,0,0,0,1,0,0
1,CCU → IXR → BBI → BLR,7h 25m,2 stops,No info,7662,1,5,5,50,13,...,0,0,0,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,19h,2 stops,No info,13882,9,6,9,25,4,...,0,0,0,1,0,0,1,0,0,0
3,CCU → NAG → BLR,5h 25m,1 stop,No info,6218,12,5,18,5,23,...,0,0,0,0,1,0,0,0,0,0
4,BLR → NAG → DEL,4h 45m,1 stop,No info,13302,1,3,16,50,21,...,0,0,0,0,0,0,0,1,0,0


In [223]:
# Additional_Info contains almost 80% no_info
# Route and Total_Stops are related to each other
df.drop(["Route", "Additional_Info"], axis = 1, inplace = True)

In [224]:
df['Total_Stops'].value_counts().index

Index(['1 stop', 'non-stop', '2 stops', '3 stops', '4 stops'], dtype='object')

In [225]:
# Replace "Total stop" values
def replace_value(x):
    if x == '1 stop':
        x = 1
    elif x == '2 stops':
        x = 2
    elif x == '3 stops':
        x = 3
    elif x == '4 stops':
        x = 4
    else:
        x = 0

    return x

df['Total_Stops'] = df['Total_Stops'].apply(replace_value)

In [226]:
df['Duration']

0         2h 50m
1         7h 25m
2            19h
3         5h 25m
4         4h 45m
          ...   
9996     10h 25m
9997     11h 45m
9998      2h 50m
9999      2h 50m
10000    13h 30m
Name: Duration, Length: 10000, dtype: object

In [227]:
# duration = list(df['Duration'])
# for i in range(len(duration)):
#     if len(duration[i].split()) != 2:    # Check if duration contains only hour or mins
#         if "h" in duration[i]:
#             duration[i] = duration[i].strip() + " 0m"   # Adds 0 minute
#         else:
#             duration[i] = "0h " + duration[i]           # Adds 0 hour
# # Prepare separate duration_hours and duration_mi

In [228]:
duration = list(df['Duration'])
duration_min = list()
duration_hour = list()
for i in range(len(duration)):
    a = duration[i].split()
    if len(a) == 2:
        duration_hour.append(a[0].rstrip('h'))
        duration_min.append(a[1].rstrip('m'))
    else:
        if 'h' in duration[i]:
            duration_hour.append(duration[i].rstrip('h'))
            duration_min.append(0)
        else:
            duration_min.append(duration[i].rstrip('m'))
            duration_hour.append(0)

In [229]:
df['Duration_mins'] = duration_min
df['Duration_hours'] = duration_hour
df.drop('Duration', axis=1, inplace=True)

In [230]:
df.columns

Index(['Total_Stops', 'Price', 'journey_day', 'journey_month', 'dep_hour',
       'dep_min', 'Arrival_hour', 'Arrival_min', 'Airline_Air India',
       'Airline_GoAir', 'Airline_IndiGo', 'Airline_Jet Airways',
       'Airline_Multiple carriers', 'Airline_SpiceJet', 'Airline_Vistara',
       'Airline_other', 'Source_Chennai', 'Source_Delhi', 'Source_Kolkata',
       'Source_Mumbai', 'Destination_Cochin', 'Destination_Delhi',
       'Destination_Hyderabad', 'Destination_Kolkata', 'Duration_mins',
       'Duration_hours'],
      dtype='object')

Fit Model

In [231]:
# Split data into training and test
x = df.drop('Price', axis=1, )
y = df['Price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [232]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor()
rf_reg.fit(x_train, y_train)

RandomForestRegressor()

In [233]:
# Calculate prediction
y_pred = rf_reg.predict(x_test)

Feature Importance

In [234]:
# Feature Importance
# Calculate feature importances using permutation importance
perm = PermutationImportance(rf_reg, random_state=42).fit(x_train, y_train)

In [236]:
# Display feature importances using ELI5
feature_names = ['Total_Stops', 'journey_day', 'journey_month', 'dep_hour',
 'dep_min', 'Arrival_hour', 'Arrival_min', 'Airline_Air India', 'Airline_GoAir', 'Airline_IndiGo', 'Airline_Jet Airways',
 'Airline_Multiple carriers', 'Airline_SpiceJet', 'Airline_Vistara', 'Airline_other', 'Source_Chennai', 'Source_Delhi',
 'Source_Kolkata', 'Source_Mumbai', 'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad', 'Destination_Kolkata',
 'Duration_mins', 'Duration_hours']
eli5.show_weights(perm, feature_names = feature_names)

Weight,Feature
0.7686 ± 0.0382,Duration_hours
0.4467 ± 0.0793,journey_day
0.3816 ± 0.0187,Airline_Jet Airways
0.2715 ± 0.0179,journey_month
0.2129 ± 0.0270,Destination_Delhi
0.1475 ± 0.0154,Total_Stops
0.0937 ± 0.0174,Arrival_hour
0.0644 ± 0.0166,Airline_other
0.0607 ± 0.0022,dep_min
0.0572 ± 0.0033,Airline_Multiple carriers


Model Performance

In [237]:
print('Model Performance on Training Set:', round(rf_reg.score(x_train, y_train)*100,2))
print('Model Performance on Test Set:', round(rf_reg.score(x_test, y_test)*100,2))

Model Performance on Training Set: 95.58
Model Performance on Test Set: 83.02


In [238]:
# Model Error Values
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

MAE: 1166.3244063943002
MSE: 3687784.6278926414
RMSE: 1920.3605463278611


Save Model

In [239]:
import pickle
# open a file, where you ant to store the data
file = open('c1_flight_rf.pkl', 'wb')
# dump information to that file
pickle.dump(rf_reg, file)

In [240]:
x_train.columns

Index(['Total_Stops', 'journey_day', 'journey_month', 'dep_hour', 'dep_min',
       'Arrival_hour', 'Arrival_min', 'Airline_Air India', 'Airline_GoAir',
       'Airline_IndiGo', 'Airline_Jet Airways', 'Airline_Multiple carriers',
       'Airline_SpiceJet', 'Airline_Vistara', 'Airline_other',
       'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Duration_mins', 'Duration_hours'],
      dtype='object')